In [2]:
import pandas as pd
import monpa
from sklearn.feature_extraction.text import TfidfVectorizer

+---------------------------------------------------------------------+
  Welcome to MONPA: Multi-Objective NER POS Annotator for Chinese
+---------------------------------------------------------------------+
已找到 model檔。Found model file.


In [4]:

train_data = pd.read_csv("./dataset/train_data.csv")
test_data = pd.read_csv("./dataset/test_data.csv")
if "segmented_content" in train_data:
    train_data = train_data.drop(columns=["segmented_content"])
if "segmented_content" in test_data:
    test_data = test_data.drop(columns=["segmented_content"])

train_data.head()

,id,post_time,content,rise,post_id
0,1648702747917_1_NEW_F02UT,2022-03-30,看你要做多久\n短期就追高殺低（找會波動的標地\n久的就逢低買進（找穩定的標地\n\n聽說是這樣,-1.0,3795
1,1646585147451_F01,2022-03-07,個人比較喜歡車<BR>可能會拿個3 400萬最多不超過500買吧<BR>或是低價買入有V12...,-1.0,69096
2,1656648774271_F01,2022-07-01,確實很慘，但，不意外,-1.0,79682
3,1683862733956_mobileR,2023-05-12,天上不會掉餡餅，一般情況下掉的都是鳥屎,1.0,112863
4,1691022982268_F0DCU,2023-08-02,會賠到8%證明你完全不知道自己在幹嘛,-1.0,54783


In [6]:
monpa.use_gpu(True)
stopwords = [line.rstrip()
             for line in open('./stopwords.txt', encoding='utf8')]
filtered_train_data = train_data.fillna('')
filtered_train_data['content'].str.strip()
filtered_test_data = test_data.fillna('')
filtered_test_data['content'].str.strip()
vectorizer = TfidfVectorizer(
    use_idf=True, stop_words=stopwords, tokenizer=monpa.cut, max_features=feature_dim)
X = vectorizer.fit_transform(
    pd.concat([filtered_train_data["content"], filtered_test_data["content"]]))

/home/namwoam/.local/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/home/namwoam/.local/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['[', ']', 'a', '一點', '下', '下去', '不可', '不斷', '主義', '之間', '二話', '些', '亦', '亮話', '人意', '今', '令', '以來', '以外', '以後', '位', '何樂', '使', '例外', '保留', '倒', '假', '偶', '充', '兒', '全', '兩', '具體', '再', '凡', '分批', '刻', '刻間', '前', '加', '勿', '卻', '口兒', '古', '只', '叮', '呆呆', '呼', '唷', '問', '啪', '喔', '噠', '嚴', '外', '夠', '大張', '大面兒', '天', '天窗', '好', '少', '巧', '差', '已', '年', '年覆', '度', '式', '彈指', '得及', '忽', '恰', '情', '慢', '慣', '成', '或少', '手段', '打開', '抗拒', '抵', '挨', '挨家', '挨戶', '挨門', '接連', '換句', '擇', '敞開', '新', '旗鼓', '日', '日覆', '早', '時', '晚', '暗地', '會兒', '末', '樂乎', '樣', '樣子', '權', '次', '止', '正', '歸根', '每刻', '每時'

In [7]:
document_vector = pd.DataFrame(
    X.toarray(), columns=vectorizer.get_feature_names_out())
label = pd.concat(
    [filtered_train_data["rise"], filtered_test_data["rise"]], ignore_index=True).astype(int)
post_id = pd.concat(
    [filtered_train_data["post_id"], filtered_test_data["post_id"]], ignore_index=True)
document_type = pd.concat(
    [pd.Series(["train"]*5000),  pd.Series(["test"]*1000)], ignore_index=True)
document_vector["label"] = label
document_vector["post_id"] = post_id
document_vector["type"] = document_type
document_vector.head()

,,,,,,,,dcard,https,0050,...,風險,高,鴻海,點,️,．,／,label,post_id,type
0,0.349278,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,-1,3795,train
1,0.222534,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.119027,0.0,0.0,0.0,0.0,0.0,0.0,-1,69096,train
2,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,-1,79682,train
3,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1,112863,train
4,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,-1,54783,train


In [8]:
document_vector.to_csv("document_vector.csv")
